In [ ]:
!pip install SNgramExtractor
!pip install spacy
!python -m spacy download en_core_web_lg
from SNgramExtractor import SNgramExtractor
from nltk.corpus import wordnet as wn
import spacy
import nltk
import re
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# Preparation & Processing Corpora

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import re
from tqdm import tqdm
import pandas as pd

Mounted at /content/drive


**NB!** Для повторения кода все необходимые материалы, а именно первоначальные корпуса, должны находится на гугл-диске (если не говорится иное / размер корпуса позволяет загрузить его на диск), который будут подключать. Все ссылки на использованные материалы представлены в соотвестсующем подразделе.

## Leipzig Corpora Collection (News discourse)

Source: Leipzig Corpopa Collection - https://wortschatz.uni-leipzig.de/en/download

In [ ]:
news_corpus_2020 = open(r'/content/drive/MyDrive/Colab Notebooks/eng_news_2020_1M-sentences.txt', encoding='utf-8', errors='ignore').read().split('\t')
news_corpus_2019 = open(r'/content/drive/MyDrive/Colab Notebooks/eng_news_2019_1M-sentences.txt', encoding='utf-8', errors='ignore').read().split('\t')
news_corpus_2018 = open(r'/content/drive/MyDrive/Colab Notebooks/eng_news_2018_1M-sentences.txt', encoding='utf-8', errors='ignore').read().split('\t')
news_corpus_2017 = open(r'/content/drive/MyDrive/Colab Notebooks/eng_news_2017_1M-sentences.txt', encoding='utf-8', errors='ignore').read().split('\t')
news_corpus_2016 = open(r'/content/drive/MyDrive/Colab Notebooks/eng_news_2016_1M-sentences.txt', encoding='utf-8', errors='ignore').read().split('\t')
news_corpus_2015 = open(r'/content/drive/MyDrive/Colab Notebooks/eng_news_2015_1M-sentences.txt', encoding='utf-8', errors='ignore').read().split('\t')

In [ ]:
def preprocessing_news(corpus):
  clean_corpus = []
  for sentence in tqdm(corpus[2:]): #первые два предложения в корпусе идут не информативными, поэтому начинаем с 3
    sentence = sentence.lower()
    sentence = re.sub(r'[\n]\d+','', sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,;…0-9]+", " ", sentence)
    clean_corpus.append(sentence)
  return clean_corpus

In [ ]:
list_of_news_corpora = [news_corpus_2015, news_corpus_2016, news_corpus_2017, news_corpus_2018, news_corpus_2019, news_corpus_2020]
news_data = pd.DataFrame(columns=['sentences'])
for corpus in list_of_news_corpora:
  corpus = preprocessing_news(corpus)
  data = pd.DataFrame(columns=['sentences'], data=corpus)
  news_data = news_data.append(data)

100%|██████████| 999999/999999 [00:13<00:00, 73996.36it/s]


In [ ]:
news_data.to_csv('news_corpus.csv') #сохраняем все обработанные корпуса для удобства применения на них после алгоритма поиска неморфологического пассива

## ACL Anthology Papers (Scientific discourse)

Научный дискурс представлен двумя корпуса в связи с тем, что было трудно найти корпус, соизмеримый по размерам с политическим и научным корпусами. Даже использование двух корпусов не помогло, и научный корпус является самым маленьким из тех, что используются в настоящей исследовательской работе.

Source: https://www.kaggle.com/datasets/takahirokubo0/acl-anthology-papers?select=emnlp-2016.csv

In [ ]:
scientific_corpus_2016 = pd.read_csv('drive/MyDrive/Colab Notebooks/emnlp-2016.csv')
scientific_corpus_2017 = pd.read_csv('drive/MyDrive/Colab Notebooks/emnlp-2017.csv')
scientific_corpus_2018 = pd.read_csv('drive/MyDrive/Colab Notebooks/emnlp-2018.csv')

In [ ]:
def preprocessing_scientific_ACL(corpus):
  clean_corpus = []
  for text in tqdm(corpus['abstract']):
    if pd.isnull(text) != True:
      text = text.split('.')
      for sentence in text:
        sentence = sentence.lower()
        sentence = re.sub(r"[^a-zA-Z?.!,;…0-9]+", " ", sentence)
        clean_corpus.append(sentence)
  return clean_corpus

In [ ]:
list_of_scientific_corpora = [scientific_corpus_2016, scientific_corpus_2017, scientific_corpus_2018]
scientific_data = pd.DataFrame(columns=['sentences'])
for corpus in list_of_scientific_corpora:
  corpus = preprocessing_scientific_ACL(corpus)
  data = pd.DataFrame(columns=['sentences'], data=corpus)
  scientific_data = scientific_data.append(data)

100%|██████████| 550/550 [00:00<00:00, 14525.68it/s]


##PubMed (Scientific corpus)

Source: https://github.com/Franck-Dernoncourt/pubmed-rct

In [ ]:
pub_med = pd.read_csv('drive/MyDrive/Colab Notebooks/pub_med.csv')

In [ ]:
def processing_scientific_PubMed(corpus):
  clean_corpus = []
  for sentence in tqdm(corpus):
    sentence = sentence.lower()
    sentence = re.sub(r"[^a-zA-Z?.!,;…0-9]+", " ", sentence)
    clean_corpus.append(sentence)
  return clean_corpus

In [ ]:
corpus = processing_scientific_PubMed(pub_med['text'])
data = pd.DataFrame(columns=['sentences'], data=corpus)
scientific_data = scientific_data.append(data)

100%|██████████| 180040/180040 [00:01<00:00, 97697.97it/s]


In [ ]:
scientific_data.to_csv('scientific_corpus.csv')

## Books (Fiction corpus)

Source: https://github.com/soskek/bookcorpus

Часть кода ниже была запущена в Jupyter Notebook для работы с локальными файлами компьютера. Выгрузка файлов на гугл диск была бы не возможной из-за размера корпуса.

In [ ]:
import os

In [ ]:
def create_book_corpus(link_to_the_corpus):
    dict_for_data = {'text': []}
    data = pd.DataFrame(dict_for_data)
    
    for filename in os.listdir(link_to_the_corpus):
        with open(os.path.join(link_to_the_corpus, filename), 'r') as f:
            text = f.read()
            new_row = {'text': text}
        data = data.append(new_row, ignore_index=True)

    return data

book_corpus = create_book_corpus('/Users/elena.kislyanicina/Downloads/books1/BooksCorpus/')

In [ ]:
def preprocessing_fiction(corpus):
    clean_corpus = []
    clean_text = []
    for text in tqdm(corpus):
        text = sent_tokenize(text)
        for sentence in text:
            sentence = sentence.lower()
            sentence = re.sub(r"[^a-zA-Z?.!,;…0-9]+", " ", sentence)
            clean_text.append(sentence)
    clean_corpus.append(clean_text)
    return clean_corpus

In [ ]:
clean_book_corpus = preprocessing_fiction(books_corpus)

100%|██████████| 128/128 [00:25<00:00,  4.95it/s]


In [ ]:
fiction_data = pd.DataFrame(columns=['sentences'])

for text in tqdm(clean_book_corpus):
  data = pd.DataFrame(columns=['sentences'], data=text)
  fiction_data = fiction_data.append(data)

100%|██████████| 128/128 [02:20<00:00,  1.10s/it]


In [ ]:
fiction_data.to_csv('fiction_corpus.csv')

##Parliamentary corpora (Political corpus)

Source: https://www.clarin.eu/resource-families/parliamentary-corpora

Код по сбору политического корпуса также был запущен в Jupyter Notebook, т.к. был необходим доступ к локальной файловой системе. Выгрузка файлов на гугл диск была бы не возможной из-за размера корпуса.

In [ ]:
!pip install conllu
from conllu import parse_incr

In [ ]:
def create_political_corpus(link_to_the_corpus):
    dict_for_data = {'text': []}
    data = pd.DataFrame(dict_for_data)
    
    for filename in tqdm(os.listdir(link_to_the_corpus)):
        if filename.endswith(".conllu"):
            with open(os.path.join(link_to_the_corpus, filename), 'r', encoding="utf-8") as f:
                political_corpus = []
                for tokenlist in parse_incr(f):
                    list_for_sentence = []
                    string = ''
                    for word in tokenlist:
                        string += word['form'] + ' '
                    political_corpus.append(string)
                new_row = {'text': political_corpus}
            data = data.append(new_row, ignore_index=True)

    return data

In [ ]:
political_corpus_2015 = create_political_corpus('/Users/elena.kislyanicina/Downloads/ParlaMint-GB.ana/ParlaMint-GB.conllu/2015/')
political_corpus_2016 = create_political_corpus('/Users/elena.kislyanicina/Downloads/ParlaMint-GB.ana/ParlaMint-GB.conllu/2016/')
political_corpus_2017 = create_political_corpus('/Users/elena.kislyanicina/Downloads/ParlaMint-GB.ana/ParlaMint-GB.conllu/2017/')
political_corpus_2018 = create_political_corpus('/Users/elena.kislyanicina/Downloads/ParlaMint-GB.ana/ParlaMint-GB.conllu/2018/')
political_corpus_2019 = create_political_corpus('/Users/elena.kislyanicina/Downloads/ParlaMint-GB.ana/ParlaMint-GB.conllu/2019/')
political_corpus_2020 = create_political_corpus('/Users/elena.kislyanicina/Downloads/ParlaMint-GB.ana/ParlaMint-GB.conllu/2020/')
political_corpus_2021 = create_political_corpus('/Users/elena.kislyanicina/Downloads/ParlaMint-GB.ana/ParlaMint-GB.conllu/2021/')

In [ ]:
def preprocessing_political(corpus):
    clean_corpus = []
    for text in tqdm(corpus['text']):
        for sentence in text:
            sentence = sentence.lower()
            sentence = re.sub(r"[^a-zA-Z?.!,;…0-9]+", " ", sentence)
            clean_corpus.append(sentence)
    return clean_corpus

In [ ]:
list_of_political_corpora = [political_corpus_2015, political_corpus_2016, political_corpus_2017, political_corpus_2018, political_corpus_2019, political_corpus_2020, political_corpus_2021]

political_data = pd.DataFrame(columns=['sentences'])

for corpus in list_of_political_corpora:
    corpus = preprocessing_political(corpus)
    data = pd.DataFrame(columns=['sentences'], data=corpus)
    political_data = political_data.append(data)

100%|██████████| 2872/2872 [00:00<00:00, 81207.81it/s]


In [ ]:
political_data.to_csv('political_corpus.csv')

# Finite constructions

In [ ]:
def finite_constructions(corpus):
    dict_for_data = {'INDEX_OF_SENTENCE': [], 'SENTENCE': [], 'CONSTRUCTION': [], 'TYPE_OF_CONSTRUCTION': []}
    data = pd.DataFrame(dict_for_data)
  
    for index, sentence in enumerate(tqdm(corpus)):
        SNgram_pos = SNgramExtractor(str(sentence),meta_tag='pos',trigram_flag='yes',nlp_model=None)
        output_pos = SNgram_pos.get_SNgram()
        SNgram_dep = SNgramExtractor(str(sentence),meta_tag='dep',trigram_flag='yes',nlp_model=None)
        output_dep = SNgram_dep.get_SNgram()
    
        for sn_gram_pos, sn_gram_dep in zip(output_pos['SNBigram'].split(), output_dep['SNBigram'].split()):
            
            if 'NOUN' in sn_gram_pos and 'VERB' in sn_gram_pos and 'nsubj' in sn_gram_dep and 'nsubjpass' not in sn_gram_dep:
                noun = sn_gram_pos.split('_')[2]
                try:
                    wn.synset('{}.n.01'.format(noun))
                except: 
                    continue
                else:
                    word = wn.synset('{}.n.01'.format(noun))
                    concrete = word.wup_similarity(wn.synset('concrete.n.01'))
                    abstract = word.wup_similarity(wn.synset('abstract.n.01'))
                    if concrete > abstract:
                        boolean_answer1 = wn.synset('person.n.01') in word.lowest_common_hypernyms(wn.synset('person.n.01'))
                        boolean_answer2 = wn.synset('people.n.01') in word.lowest_common_hypernyms(wn.synset('people.n.01'))
                        boolean_answer5 = wn.synset('animal.n.01') in word.lowest_common_hypernyms(wn.synset('animal.n.01'))
                        if boolean_answer1 == False and boolean_answer2 == False and boolean_answer5 == False:
                            new_row = {'INDEX_OF_SENTENCE': index, 'SENTENCE': sentence, 'CONSTRUCTION': sn_gram_pos, 'TYPE_OF_CONSTRUCTION': 'finite constructions'}
                            data = data.append(new_row, ignore_index=True)
    return data

# Retroactive infinitive

In [ ]:
def retroactive_infinitive(corpus):
    dict_for_data = {'INDEX_OF_SENTENCE': [], 'SENTENCE': [], 'CONSTRUCTION': [], 'TYPE_OF_CONSTRUCTION': []}
    data = pd.DataFrame(dict_for_data)
  
    nlp = spacy.load("en_core_web_sm")

    for index, sentence in enumerate(tqdm(corpus)):
        SNgram_pos = SNgramExtractor(str(sentence),meta_tag='pos',trigram_flag='yes',nlp_model=None)
        output_pos = SNgram_pos.get_SNgram()
        SNgram_dep = SNgramExtractor(str(sentence),meta_tag='dep',trigram_flag='yes',nlp_model=None)
        output_dep = SNgram_dep.get_SNgram()

        infinitive_verb = '*'    

        for sn_gram in output_pos['SNBigram'].split():
            if '_VERB_to_PART' in sn_gram:
                infinitive_verb = sn_gram.split('_')[0]
                doc = nlp(infinitive_verb)
                infinitive_verb = doc[0].lemma_

                for sn_gram_pos, sn_gram_dep in zip(output_pos['SNBigram'].split(), output_dep['SNBigram'].split()):
        
                    checker1 = re.findall(r'(?:is|are|am|was|were|will)_ROOT_{}_xcomp'.format(infinitive_verb), sn_gram_dep)
                    checker2 = re.findall(r'(?:is|are|am|was|were|will)_conj_{}_xcomp'.format(infinitive_verb), sn_gram_dep)

                    if infinitive_verb in sn_gram_pos and 'NOUN' in sn_gram_pos.split('_')[1] and 'VERB' in sn_gram_pos and 'relcl' in sn_gram_dep and 'dobj' not in sn_gram_dep and 'order' not in sn_gram_pos:
                        new_row = {'INDEX_OF_SENTENCE': index, 'SENTENCE': sentence, 'CONSTRUCTION': sn_gram_pos, 'TYPE_OF_CONSTRUCTION': 'retroactive infinitive'}
                        data = data.append(new_row, ignore_index=True)
                    elif checker1 != [] or checker2 != []:
                        
                        checker3 = re.findall(r'(?:is|are|am|was|were|will) [a-z]+ to {}'.format(infinitive_verb), sentence)
                        checker4 = re.findall(r'(?:is|are|am|was|were|will) [a-z]+ [a-z]+ to {}'.format(infinitive_verb), sentence)
                        
                        if checker3 != [] or checker4 != []:
                            new_row = {'INDEX_OF_SENTENCE': index, 'SENTENCE': sentence, 'CONSTRUCTION': sn_gram_pos, 'TYPE_OF_CONSTRUCTION': 'retroactive infinitive'}
                            data = data.append(new_row, ignore_index=True)
                    elif infinitive_verb in sn_gram_pos and 'ADJ' in sn_gram_pos.split('_')[1] and 'VERB' in sn_gram_pos and 'xcomp' in sn_gram_dep and 'able' not in sn_gram_pos:
                        new_row = {'INDEX_OF_SENTENCE': index, 'SENTENCE': sentence, 'CONSTRUCTION': sn_gram_pos, 'TYPE_OF_CONSTRUCTION': 'retroactive infinitive'}
                        data = data.append(new_row, ignore_index=True)
    return data

# Gerundial Constructions

In [ ]:
def gerundial_construction(corpus):
  dict_for_data = {'INDEX_OF_SENTENCE': [], 'SENTENCE': [], 'CONSTRUCTION': [], 'TYPE_OF_CONSTRUCTION': []}
  data = pd.DataFrame(dict_for_data)
  
  for index, sentence in enumerate(tqdm(corpus)):
    SNgram_pos = SNgramExtractor(sentence,meta_tag='pos',trigram_flag='yes',nlp_model=None)
    output_pos = SNgram_pos.get_SNgram()

    for sn_gram in output_pos['SNBigram'].split():
      if 'worth_ADJ' in sn_gram and 'VERB' in sn_gram:
        new_row = {'INDEX_OF_SENTENCE': index, 'SENTENCE': sentence, 'CONSTRUCTION': sn_gram, 'TYPE_OF_CONSTRUCTION': 'gerundial construction'}
        data = data.append(new_row, ignore_index=True)

      checker = re.findall(r'(?:bear|deserve|need|require|want)[a-z]*_VERB_[a-z]+ing_VERB', sn_gram)
      if checker != []:
        new_row = {'INDEX_OF_SENTENCE': index, 'SENTENCE': sentence, 'CONSTRUCTION': sn_gram, 'TYPE_OF_CONSTRUCTION': 'gerundial construction'}
        data = data.append(new_row, ignore_index=True)

  return data

# Lexical passive

In [ ]:
def lexical_passive(corpus):
    dict_for_data = {'INDEX_OF_SENTENCE': [], 'SENTENCE': [], 'CONSTRUCTION': [], 'TYPE_OF_CONSTRUCTION': []}
    data = pd.DataFrame(dict_for_data)
  
    for index, sentence in enumerate(tqdm(corpus)):
        SNgram_pos = SNgramExtractor(str(sentence),meta_tag='pos',trigram_flag='yes',nlp_model=None)
        output_pos = SNgram_pos.get_SNgram()
        SNgram_dep = SNgramExtractor(str(sentence),meta_tag='dep',trigram_flag='yes',nlp_model=None)
        output_dep = SNgram_dep.get_SNgram()

        for sn_gram_pos, sn_gram_dep in zip(output_pos['SNBigram'].split(), output_dep['SNBigram'].split()):
            lexical_passive_verb = re.findall(r'(?:receiv|suffer|underg|underw)[a-z]*_VERB_[a-z]+_(?:NOUN|PRON|PROPN|NUM)', sn_gram_pos)

            if lexical_passive_verb != [] and 'nsubj' in sn_gram_dep:
                new_row = {'INDEX_OF_SENTENCE': index, 'SENTENCE': sentence, 'CONSTRUCTION': sn_gram_pos, 'TYPE_OF_CONSTRUCTION': 'lexical passive'}
                data = data.append(new_row, ignore_index=True)
    return data

#Final corpus of passive-like constructions

Данная часть запускалась в Jupyter Notebook, т.к. требовалось значительное время для обработки каждого первоначального корпуса.

In [ ]:
from sklearn.utils import shuffle

In [ ]:
news_corpus = pd.read_csv('/Users/elena.kislyanicina/Downloads/news_corpus.csv')
scientific_corpus = pd.read_csv('/Users/elena.kislyanicina/Downloads/scientific_corpus.csv')
fiction_corpus = pd.read_csv('/Users/elena.kislyanicina/Downloads/fiction_corpus.csv')
political_corpus = pd.read_csv('/Users/elena.kislyanicina/Downloads/political_corpus.csv')

In [ ]:
def find_passive_like_constructions(corpus):
    
    f_constructions = finite_constructions(corpus)
    r_i_constructions = retroactive_infinitive(corpus)
    g_construction = gerundial_construction(corpus)
    l_p_constructions = lexical_passive(corpus)
    
    frames = [f_constructions, r_i_constructions, g_construction, l_p_constructions]
    
    data = pd.concat(frames)
    
    return data

In [ ]:
scientific_discourse = find_passive_like_constructions(scientific_corpus['sentences'])

news_corpus_shuffle = shuffle(news_corpus)
news_discourse = find_passive_like_constructions(news_corpus_shuffle['sentences'][:200001])

political_corpus_shuffle = shuffle(political_corpus)
political_discourse = find_passive_like_constructions(political_corpus_shuffle['sentences'][:200001])

fiction_corpus_shuffle = shuffle(fiction_corpus)
fiction_discourse = find_passive_like_constructions(fiction_corpus_shuffle['sentences'][:200001])

In [ ]:
scientific_discourse.to_csv('scientific_discourse.csv')
news_discourse.to_csv('news_discourse.csv')
political_discourse.to_csv('political_discourse.csv')
fiction_discourse.to_csv('fiction_discourse.csv')

## Additional processing

Дополнительные фильтры, которые были использованы, чтобы снизить процент шума в подкорпусах

In [ ]:
news_d = pd.read_csv('/Users/elena.kislyanicina/Downloads/news_discourse.csv')
scientific_d = pd.read_csv('/Users/elena.kislyanicina/Downloads/scientific_discourse.csv')
fiction_d = pd.read_csv('/Users/elena.kislyanicina/Downloads/fiction_discourse.csv')
political_d = pd.read_csv('/Users/elena.kislyanicina/Downloads/political_discourse.csv')

In [ ]:
def filter_1(corpus): #был нужен, т.к. из-за несовсем корректной очистки корпуса было много конструкций, в которых части отрицания считались за существительное
    check_list = []
    for construction in corpus['CONSTRUCTION']:
        if construction.split('_')[2] == 'i' or len(construction.split('_')[2]) > 1: 
            check_list.append(True)
        else:
            check_list.append(False)
    corpus['check'] = check_list
    corpus = corpus.drop(np.where(corpus['check'] == False)[0])
    corpus.drop(columns = ['check'],axis = 1, inplace=True)
    
    return corpus

In [ ]:
news_d = filter_1(news_d)
scientific_d = filter_1(scientific_d)
political_d = filter_1(political_d)
fiction_d = filter_1(fiction_d)

In [ ]:
def filter_2(corpus): # был нужен для того, чтобы избавится от предложений в которых был явно использован классический пассив, отсылающий к деятелю
    
    check_list = []
    for construction, type_of_construction in zip(corpus['CONSTRUCTION'], corpus['TYPE_OF_CONSTRUCTION']):
    
        if (construction.split('_')[0] == 'ready' and type_of_construction == 'retroactive infinitive') or (construction.split('_')[2] == 'see' and type_of_construction == 'retroactive infinitive') or (construction.split('_')[2] == 'hear' and type_of_construction == 'retroactive infinitive') or (construction.split('_')[2] == 'say' and type_of_construction == 'retroactive infinitive'):
            check_list.append(True)
        else:
            check_list.append(False)
    
    corpus['check'] = check_list
    corpus = corpus.drop(np.where(corpus['check'] == True)[0])
    corpus.drop(columns = ['check'],axis = 1, inplace=True)
    
    return corpus

In [ ]:
news_d = filter_2(news_d)
scientific_d = filter_2(scientific_d)
political_d = filter_2(political_d)
fiction_d = filter_2(fiction_d)

In [ ]:
news_d['DISCOURSE'] = ['news']*19829 # добавление финальных меток дискурса перед склейкой в один корпус
political_d['DISCOURSE'] = ['political']*19842
scientific_d['DISCOURSE'] = ['scientific']*18091
fiction_d['DISCOURSE'] = ['fiction']*13964

In [ ]:
final_corpus = pd.concat([fiction_d, scientific_d, news_d, political_d], axis=0)

In [ ]:
final_corpus.to_csv('passive_like_construction_corpus.csv')

Финальный корпус доступен по ссылке: https://docs.google.com/spreadsheets/d/1dqtiHbnmVuNSn6gf0LQ97gwTIQ_HbJFi19MJQSzIS8o/edit?usp=sharing 